In [1]:
!pip install rdkit
!pip install mordred

'pip' is not recognized as an internal or external command,
operable program or batch file.
'pip' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
from rdkit import Chem, DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import AllChem
from rdkit.Chem.Pharm2D import Generate
from rdkit.Chem.Pharm2D.SigFactory import SigFactory
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.Pharm2D.SigFactory import SigFactory
from rdkit.Chem.Pharm2D import Generate
import pandas as pd
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import DataStructs
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
fdefName = 'MinimalFeatures.fdef'
featFactory = ChemicalFeatures.BuildFeatureFactory(fdefName)
sigFactory = SigFactory(featFactory, minPointCount=2, maxPointCount=3, trianglePruneBins=False)
sigFactory.SetBins([(0, 2), (2, 5), (5, 8)])
sigFactory.Init()

In [5]:
def calc_tanimoto(fp1, fp2):
    tc = DataStructs.TanimotoSimilarity(fp1, fp2)
    return round(tc,2)

In [ ]:
train = pd.read_csv('TrainQSAR.csv')
test= pd.read_csv('TestQSAR.csv')

In [ ]:
for index, row in train.iterrows():
    smiles = row['SMILES']
    mol = Chem.MolFromSmiles(smiles)
    fp = Generate.Gen2DFingerprint(mol,sigFactory)
    train.at[index, 'fp'] = fp

In [ ]:
for indexs, rows in test.iterrows():
    smiless = rows['SMILES']
    mols = Chem.MolFromSmiles(smiless)
    fps = Generate.Gen2DFingerprint(mols,sigFactory)
    test.at[indexs, 'fp'] = fps

In [ ]:
import numpy as np
y = train['fp']
Y = pd.DataFrame(y)
x = test['fp']
X = pd.DataFrame(x)
# Create an empty matrix to store the Tanimoto coefficients
matrix = np.zeros((len(X), len(Y)))

for i, row_x in X.iterrows():
    smiles_x = row_x['fp']
    for j, row_y in Y.iterrows():
        smiles_y = row_y['fp']
        coefficient = calc_tanimoto(smiles_x, smiles_y)
        matrix[i, j] = coefficient

In [ ]:
matrix_df = pd.DataFrame(matrix)
plt.figure(figsize=(8, 6))  
sns.heatmap(matrix_df, cmap='YlOrRd')
plt.show()